In [181]:
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
%matplotlib inline 

In [205]:
def scrape_wikpedia_wc_players(year):
    url = "https://en.wikipedia.org/wiki/{}_FIFA_World_Cup_squads".format(year)
    time.sleep(5)
    my_page = requests.get(url)
    if my_page.status_code == 200:
        soup = BeautifulSoup(my_page.content, "lxml")
        tables = soup.find_all('table',{'class':'sortable wikitable plainrowheaders'})
        countries = soup.find_all('li', {'class': 'toclevel-2'})[:32]
        players_df = pd.DataFrame(columns = ['Player', 'Country', 'Pos', 'Birthday', 'Age', 'Caps', 'Club', 'Club Country'])
        for i, table in enumerate(tables):
            #first result would be column names
            country = countries[i].find('span', {'class': 'toctext'}).text
            players = table.find_all('tr')[1:]
            for player in players:
                name = player.find('th').find('a').text
                td_tags = player.find_all('td')
                position = td_tags[1].find('a').text
                birthday = td_tags[2].text[2:12]
                age = td_tags[2].text[-4:-2]
                caps = td_tags[3].text.replace('\n', "")
                #Only 2018 has goals, so do not keep it.
#                 goals = td_tags[4].text.replace('\n', "")
                #Last element is club
                club = td_tags[-1].find_all('a')[1].text
                club_country = td_tags[-1].find('a')['title']
                row = pd.DataFrame([[name, country, position, birthday, age, caps, club, club_country]],\
                                   columns = ['Player', 'Country', 'Pos', 'Birthday', 'Age', 'Caps', 'Club', 'Club Country'])
                players_df = players_df.append(row)
        players_df = players_df.reset_index(drop = True)
        return players_df

In [206]:
wc_players_2018 = scrape_wikpedia_wc_players(2018)

In [207]:
wc_players_2018.head()

,Player,Country,Pos,Birthday,Age,Caps,Club,Club Country
0,Essam El-Hadary,Egypt,GK,1973-01-15,45,158,Al-Taawoun,Saudi Arabia
1,Ali Gabr,Egypt,DF,1989-01-01,29,21,West Bromwich Albion,England
2,Ahmed Elmohamady,Egypt,DF,1987-09-09,30,78,Aston Villa,England
3,Omar Gaber,Egypt,MF,1992-01-30,26,24,Los Angeles FC,United States
4,Sam Morsy,Egypt,MF,1991-09-10,26,5,Wigan Athletic,England


In [208]:
wc_players_2018.to_csv("data/wc_players_2018.csv")

In [209]:
wc_players_2014 = scrape_wikpedia_wc_players(2014)

In [210]:
wc_players_2014.head()

,Player,Country,Pos,Birthday,Age,Caps,Club,Club Country
0,Jefferson,Brazil,GK,1983-01-02,31,9,Botafogo,Brazil
1,Dani Alves,Brazil,DF,1983-05-06,31,75,Barcelona,Spain
2,Thiago Silva,Brazil,DF,1984-09-22,29,46,Paris Saint-Germain,France
3,David Luiz,Brazil,DF,1987-04-22,27,36,Chelsea,England
4,Fernandinho,Brazil,MF,1985-05-04,29,7,Manchester City,England


In [211]:
wc_players_2014.to_csv("data/wc_players_2014.csv")